In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from zipfile import ZipFile
import urllib.request
import os
import geopy.distance

from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\melis\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [3]:
url = 'https://s3.amazonaws.com/tripdata/index.html'
browser.visit(url)


In [4]:
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# Retrieve all elements that contain book information
all_a_tags = soup.find_all('a')

#all_links = [link.get_attribute('href') for link in browser.find_elements_by_tag_name('a')]

In [5]:
zip_links = []
for link in all_a_tags:
    if ('2016' in link['href']) & ('JC' in link['href']):
        zip_links.append(link['href'].replace(' ','%20'))
    if ('2017' in link['href']) & ('JC' in link['href']):        
        zip_links.append(link['href'].replace(' ','%20'))
zip_links    

['https://s3.amazonaws.com/tripdata/JC-201601-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201602-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201603-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201604-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201605-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201606-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201607-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201608-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201609-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201610-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201611-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201612-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC-201701-citibike-tripdata.csv.zip',
 'https://s3.amazonaws.com/tripdata/JC

In [6]:
def getcitybike_df(file_url):
    url = urllib.request.urlopen(file_url)
    output = open('data.zip', 'wb')    
    output.write(url.read())
    output.close()
    df = pd.read_csv('data.zip')    
    if len(df.columns) == 15:
        df.columns = ['Trip Duration (sec)'
            ,'Start Datetime'
            ,'End Datetime'
            ,'Start Station ID'
            ,'Start Station Name'
            ,'Start Latitude'
            ,'Start Longitude'
            ,'End Station ID'
            ,'End Station Name'
            ,'End Latitude'
            ,'End Longitude'
            ,'BikeID'
            ,'UserType'
            ,'BirthYear'
            ,'Gender']    

    print(f'Row count {len(df.index)}')
    
    os.remove('data.zip')

    print(file_url)
    return df

In [7]:
def getcleandata_df(df):    
    df = df.dropna(how='any').reset_index(drop=True)
            
    df['Start Datetime'] = pd.to_datetime(df['Start Datetime'])
    df['End Datetime'] = pd.to_datetime(df['End Datetime'])
    df['BirthYear'] = df.BirthYear.astype(int)
        
    df['Age'] = df['Start Datetime'].dt.year - df['BirthYear']
    
    # Gender (Zero=unknown; 1=male; 2=female)    
    df = df[ df['Start Latitude'] != 0 ]
    df = df[ df['End Latitude'] != 0 ]
    df = df[ df['Start Longitude'] != 0 ]
    df = df[ df['End Longitude'] != 0 ]
    # exclude ages > 90 years
    df = df[df['Age'] < 99]
    
    return df

In [8]:
citybikedata_df = pd.DataFrame()


In [9]:
for link in zip_links:
    df = getcitybike_df(link)
    citybikedata_df = citybikedata_df.append(df, ignore_index=True, sort=False)    

Row count 7479
https://s3.amazonaws.com/tripdata/JC-201601-citibike-tripdata.csv.zip
Row count 8250
https://s3.amazonaws.com/tripdata/JC-201602-citibike-tripdata.csv.zip
Row count 13511
https://s3.amazonaws.com/tripdata/JC-201603-citibike-tripdata.csv.zip
Row count 16342
https://s3.amazonaws.com/tripdata/JC-201604-citibike-tripdata.csv.zip
Row count 19488
https://s3.amazonaws.com/tripdata/JC-201605-citibike-tripdata.csv.zip
Row count 23947
https://s3.amazonaws.com/tripdata/JC-201606-citibike-tripdata.csv.zip
Row count 24436
https://s3.amazonaws.com/tripdata/JC-201607-citibike-tripdata.csv.zip
Row count 34149
https://s3.amazonaws.com/tripdata/JC-201608-citibike-tripdata.csv.zip
Row count 33425
https://s3.amazonaws.com/tripdata/JC-201609-citibike-tripdata.csv.zip
Row count 29611
https://s3.amazonaws.com/tripdata/JC-201610-citibike-tripdata.csv.zip
Row count 21832
https://s3.amazonaws.com/tripdata/JC-201611-citibike-tripdata.csv.zip
Row count 15114
https://s3.amazonaws.com/tripdata/JC-201

In [10]:
citybikedata_df.head()

,Trip Duration (sec),Start Datetime,End Datetime,Start Station ID,Start Station Name,Start Latitude,Start Longitude,End Station ID,End Station Name,End Latitude,End Longitude,BikeID,UserType,BirthYear,Gender
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964.0,2
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962.0,1
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962.0,2
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984.0,1
4,903,2016-01-01 01:03:20,2016-01-01 01:18:24,3195,Sip Ave,40.730743,-74.063784,3210,Pershing Field,40.742677,-74.051789,24573,Customer,NaN,0


In [11]:
citybikedata_df.dtypes

Trip Duration (sec)      int64
Start Datetime          object
End Datetime            object
Start Station ID         int64
Start Station Name      object
Start Latitude         float64
Start Longitude        float64
End Station ID           int64
End Station Name        object
End Latitude           float64
End Longitude          float64
BikeID                   int64
UserType                object
BirthYear              float64
Gender                   int64
dtype: object

In [12]:
clean_citybikedata_df = getcleandata_df(citybikedata_df)

In [13]:
clean_citybikedata_df.dtypes

Trip Duration (sec)             int64
Start Datetime         datetime64[ns]
End Datetime           datetime64[ns]
Start Station ID                int64
Start Station Name             object
Start Latitude                float64
Start Longitude               float64
End Station ID                  int64
End Station Name               object
End Latitude                  float64
End Longitude                 float64
BikeID                          int64
UserType                       object
BirthYear                       int32
Gender                          int64
Age                             int64
dtype: object

In [14]:
trip_distances = []
for index, row in clean_citybikedata_df.iterrows():    
    start_lat = row['Start Latitude']
    start_lng = row['Start Longitude']
    end_lat = row['End Latitude']
    end_lng = row['End Longitude']

    coord1 = (start_lat, start_lng)
    coord2 = (end_lat, end_lng)
    
    trip_distances.append(geopy.distance.geodesic(coord1, coord2).miles)    



In [15]:
clean_citybikedata_df['Approximate Trip Distance (mi)'] = trip_distances

In [16]:
clean_citybikedata_df.head()

,Trip Duration (sec),Start Datetime,End Datetime,Start Station ID,Start Station Name,Start Latitude,Start Longitude,End Station ID,End Station Name,End Latitude,End Longitude,BikeID,UserType,BirthYear,Gender,Age,Approximate Trip Distance (mi)
0,362,2016-01-01 00:02:52,2016-01-01 00:08:54,3186,Grove St PATH,40.719586,-74.043117,3209,Brunswick St,40.724176,-74.050656,24647,Subscriber,1964,2,52,0.506924
1,200,2016-01-01 00:18:22,2016-01-01 00:21:42,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24605,Subscriber,1962,1,54,0.253551
2,202,2016-01-01 00:18:25,2016-01-01 00:21:47,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24689,Subscriber,1962,2,54,0.253551
3,248,2016-01-01 00:23:13,2016-01-01 00:27:21,3209,Brunswick St,40.724176,-74.050656,3203,Hamilton Park,40.727596,-74.044247,24693,Subscriber,1984,1,32,0.410937
4,445,2016-01-01 01:07:45,2016-01-01 01:15:11,3186,Grove St PATH,40.719586,-74.043117,3203,Hamilton Park,40.727596,-74.044247,24510,Subscriber,1988,2,28,0.555872


In [17]:
clean_citybikedata_df.to_csv('data/citybikedata.csv', encoding='utf-8', index=False)